<a href="https://colab.research.google.com/github/AlexandreMinato/AlexandreMinato/blob/main/AnaliseOpcoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.by import By
import selenium.webdriver.common.keys
from selenium.webdriver.common.keys import Keys

In [4]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)


In [5]:
def formatar_coluna(df_formatar, colunas):
  for coluna in colunas:
    df_formatar[coluna] = df_formatar[coluna]/100 
    

In [6]:
def capturar_dados_opc(ticker, colunas_df):
    url = "https://opcoes.net.br/opcoes/bovespa/" + ticker
    wait = WebDriverWait(wd, 5000)
    time.sleep(2)
    wd.get(url)
    time.sleep(2) #AGUARDA o script do Datatable rodar
    html = wd.page_source
    soup = BeautifulSoup(html, 'lxml')
    table = soup.find_all('table', { "id" : "tblListaOpc" }) #tblListaOpc
    df_result = pd.read_html(str(table))
    df_result[0].columns = colunas_df
    df_result = df_result[0]
    df_result.set_index('ticker')
    colunas_formatar = ['strike',  'vol_implicita', 'delta', 'gamma', 'theta_R$', 'theta', 'vega']
    formatar_coluna(df_formatar=df_result, colunas=colunas_formatar)
    df_result.insert(1,'ativo', ticker) 
    return df_result  

In [7]:
posicoes = ['BBDC4','EGIE3','ITSA4','BEEF3','COGN3','BBAS3','GOAU4','CVCB3','SBSP3','EVEN3','ARZZ3','PETR4','SULA11','B3SA3','CIEL3','TRPL4','IRBR3']
colunas = ['ticker', 'tipo','FM','mod','strike','a_i_OTM','ultimo','variacao','data','num_negocios','volume_financeiro','vol_implicita','delta','gamma','theta_R$', 'theta', 'vega']
df_consolidado = pd.DataFrame(columns=colunas)

In [ ]:
for posicao in posicoes:
  df_temp = capturar_dados_opc(posicao, colunas)
  time.sleep(2)
  df_consolidado = df_consolidado.append(df_temp, ignore_index=True)

In [18]:
df_consolidado.to_csv('opcoes.csv')

In [99]:
def capturar_valor_black_scholes(tiker_name):
  url_bs = 'https://opcoes.net.br/calculadora-Black-Scholes'
  wd.get(url_bs)
  elem = wd.find_element_by_id("ticker")
  elem.clear()
  elem.send_keys(tiker_name)
  elem.send_keys(Keys.RETURN)
  elem.send_keys(Keys.ENTER)
  time.sleep(2)
  bs_ticker = wd.find_elements_by_id("ticker")[0].get_attribute('value')
  bs_strike = wd.find_elements_by_id("strike")[0].get_attribute('value')
  bs_txjuros = wd.find_elements_by_id("taxaDeJuros")[0].get_attribute('value')
  bs_cotacao = wd.find_elements_by_id("cotacaoAcao")[0].get_attribute('value')
  bs_premio = wd.find_elements_by_id("premioDaOpcao")[0].get_attribute('value')
  df2 = {'ticker': bs_ticker, 'strike': bs_strike, 'taxaDeJuros': bs_txjuros, 'cotacaoAcao': bs_cotacao, 'premioDaOpcao': bs_premio}
  return df2

In [104]:
df_bs = pd.DataFrame(columns=['ticker', 'strike', 'taxaDeJuros', 'cotacaoAcao', 'premioDaOpcao'])

In [105]:
for index, row in df_consolidado.iterrows():
  df_bs = df_bs.append(capturar_valor_black_scholes(row['ticker']), ignore_index=True)
  print(row['ticker'], row['premioDaOpcao'])

MaxRetryError: ignored

In [ ]:
df_bs.set_index('ticker')

In [ ]:
df_consolidado.set_index('ticker')

In [85]:
df_union = pd.concat([df_bs, df_consolidado], axis=1)

In [90]:
df_union.columns = ['ticker_bs','strike_bs','taxaDeJuros_bs','cotacaoAcao_bs','premioDaOpcao_bs','ticker','tipo','FM','mod','strike','a_i_OTM','ultimo','variacao','data','num_negocios','volume_financeiro','vol_implicita','delta','gamma','theta_R$','theta','vega','ativo']

In [ ]:
df_union.set_index('ticker')

In [98]:
#df_union[['ticker', 'strike_bs']]
df_union.to_csv('union.csv')